In deze opgave classificeren afbeeldingen van bloemen uit de iris dataset met behulp van een neuraal netwerk.

In [248]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Laad de dataset en maak een feature matrix X en een target vector y.

In [249]:
iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values 
y_iris = iris['species'].values

In [250]:
y_iris = y_iris.reshape(-1, 1) # maak een array van array's, dit is nodig voor de volgende stap
print(y_iris[:10])

[['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']]


Een neuraal netwerk kan niet omgaan met categorische labels zoals de namen van de planten, maar heeft getallen als uitput nodig. We hebben hier drie klassen (setosa, versicolor, virginica) daarom maken we een target vector met per label drie waarden. Een 1 voor de eerste waarde correspondeert met 'setosa', een 1 voor de 2e waade met 'versicolor' en een 1 voor de derde waarde met 'virginica'.

Doe dit met behulp van het `OneHotEncoder` object uit sklearn.

In [251]:
enc = OneHotEncoder(sparse=False)
y_iris = enc.fit_transform(y_iris)

We hebben de data nu in een geschikt formaat. Splits de data in training en test data.

In [252]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, random_state=5, train_size=0.8)

Maak een eenvoudig neuraal netwerk met 3 lagen, waarin elke neuron in een laag met elk neuron in de vorige laag verbonden is ('Dense'). 
Voeg voor de eeste laag een parameter `input_shape` toe die aangeeft hoeveel features een element in 'X' heeft.
Het aantal neuronen in de laaste laag is gelijk aan het aantal waarden in een label in `y`. 

In [253]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(4, input_shape=(4,), activation='sigmoid'))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='softmax'))

#voeg lagen toe met model.add()

Compileer en bekijk het netwerk door onderstaande code te runnen.

In [254]:
from keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_63 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_64 (Dense)             (None, 3)                 15        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________


Nu kunnen we ons model trainen met behulp van de `fit` methode.

In [255]:
# model.fit(X_train, y_train, epochs = 10, batch_size = 5)

Met `model_evaluate` kunnen we bepalen hoe goed het model werkt op de test data.

In [256]:
# test_loss, test_acc = model.evaluate(X_test, y_test)
# print(test_acc)

Experimenteer met bovenstaand model. Wat gebeurt er als je meer lagen toevoegt of een laag weghaalt of het aantal neuronen in een laag verandert? 

Probeer ook eens andere activatiefuncties dan `relu`, zoals `tanh` of `sigmoid`.

Welke invloed heeft het aantal epochs?

Als ik meer lagen toevoeg heeft het model snel de neiging om te gaan overfitten, omdat de dataset te simpel is.

Als het aantal epochs te klein is heeft het model niet genoeg tijd om te trainen. Maar als het te lang duurt dan krijg
je op een gegeven moment afnemende opbrengsten.



We kunnen dezelfde aanpak gebruiken voor andere data, bijvoorbeeld voor het herkennen van cijfers. We gebruiken de `MNist` dataset die bestaat uit plaatjes van cijfers. Ieder plaatje bestaat uit 28 x 28 pixels.

In [257]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

import warnings
warnings.filterwarnings('ignore')

In [258]:
train_images.shape

(60000, 28, 28)

In [259]:
len(train_labels)

60000

In [260]:
print(train_labels)

[5 0 4 ... 5 6 8]


Een eenvoudig neuraal netwerk heeft een array van inputwaarden nodig i.p.v. een 2D afbeelding.
We maken van de inputs (waarden tussen 0 en 255) getallen tussen 0 en 1.

In [261]:
train_images = train_images.reshape((60000, 28 * 28)) # lijst van waarden i.p.v. 2D afbeelding
train_images = train_images.astype('float32') / 255 # getallen tussen 0 en 1

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Voor elk label (een cijfer) maken we een array met 10 waarden: 9 nullen en een één, waarbij de positie van de één aangeeft om welk cijfer het gaat (One hot encoding). Dit keer gebruiken we hiervoor de `keras` functie `to_categorical`.

In [262]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


Maak nu een neuraal netwerk met twee lagen, een eerste laag met 512 neuronen en een output laag met 10 neuronen. Geef bij de eerste laag aan hoeveel inputs er zijn en wat de activatiefunctie is. De output laag heeft een `softmax` activatiefunctie.

In [263]:
model2 = Sequential()
model2.add(Dense(512, input_shape=(28*28,), activation='relu'))
model2.add(Dense(10, activation='softmax'))

compileer het netwerk.

In [264]:
model2.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_66 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


train het netwerk 5 epochs met een batch_size van 128.

In [265]:
model2.fit(train_images, train_labels, epochs=10, batch_size=1000)

Epoch 1/10
120/120 [==============================] - 10s 84ms/step - loss: 0.6248 - accuracy: 0.8095

Evalueer het netwerk op de test set.

In [266]:
model2.evaluate(test_images, test_labels)

313/313 [==============================] - 17s 55ms/step - loss: 0.1255 - accuracy: 0.9724

[0.125533327460289, 0.9724000096321106]

Experimenteer met verschillende netwerken. Varieer het aantal lagen, het aantal neuronen, de activatiefunties en het aantal epochs.